# Data cleaning, so we can build a classifiier

### **Note:** Remember to check which environment this notebook is using!

## Steps in this notebook
1. Get the packages we need (install -> import)
2. `emnist` pulls a clean dataset of handwritten characters
    1. Clean dataframes are `raw_train` and `raw_test`
    2. Each has only two columns
        1. 'image' - 28x28 pixel array images of handwritten characters
        2. 'label' - different number for class of digit: 0 -> 0, 10 -> A, 61 -> z
3. Demonstrate basic plotting of the images
4. Intentionally dirty this clean dataset
    1. Dirty dataframes are `dirty_train` and `dirty_test`
    2. Look through how the dirtying process works for ideas on how to clean up
5. Space for whatever you're going to do next
    1. Get cleaning with pandas
    2. Build classifiers with the data you've cleaned
    3. Skip data cleaning and build classifier with the raw data

# Install packages

Uncomment the followinig lines the first time running this notebook in this environment
`%conda install pandas numpy emnist matplotlib`
`%pip install emnist`

In [ ]:
# Import packages
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import emnist
from hashlib import sha1

In [ ]:
# Load the data, and reshape it into a 28x28 array

# The size of each image is 28x28 pixels
size = 28 

# Extract the training split as images and labels
image, label = emnist.extract_training_samples('byclass')

# Add columns for each pixel value (28x28 = 784 columns)
raw_train = pd.DataFrame()

# Add a column showing the label
raw_train['label'] = label

# Add a column with the image data as a 28x28 array
raw_train['image'] = list(image)


# Repeat for the test split
image, label = emnist.extract_test_samples('byclass')
raw_test = pd.DataFrame()
raw_test['label'] = label
raw_test['image'] = list(image)


In [ ]:
# We can plot individual images using matplotlib
plt.imshow(raw_train['image'][0], cmap='gray')
plt.show() # Show the plot (optional with a single image)

In [ ]:
# Get the first row for each label
firsts = raw_train.groupby('label').first().reset_index()

# Build a plot with the first image for each label
fig, ax = plt.subplots(7, 10, figsize=(10, 7))
for i in range(62):
    ax[i//10, i%10].imshow(firsts['image'][i], cmap='gray')
    ax[i//10, i%10].axis('off')
    ax[i//10, i%10].set_title(firsts['label'][i])

# Getting dirty

Now that we've got the data in good shape, let's rough it up a little.

- [x] Add `predicit` col with confidence probabilities from a previous model
- [x] Numerical:
    - [x] outlier
    - [x] out-of-bounds
- [x] Labels: missing(Null, None, "", " "), name that number, double-struck
- [x] Image: zeroed, null? dimensions?
- [x] Image: add noise
- [x] Image: flip horizonally
- [x] Duplicated rows

In [ ]:
# Now let's mess up the data a bit

# Percent of the time something dirty happens (0.1%) for each method
pct = 0.001 

# Copy the splits into new dataframes to mess up
dirty_train = raw_train.copy()
dirty_test  = raw_test.copy()

# Add a column for a hash of the images (should make it easier to compare them)
dirty_train['image_hash'] = dirty_train['image'].apply(lambda x: sha1(x.tobytes()).hexdigest())
dirty_test['image_hash']  =  dirty_test['image'].apply(lambda x: sha1(x.tobytes()).hexdigest())

# For each row, 0.1% of the time, duplicate the row
dirty_train = pd.concat([dirty_train, dirty_train.sample(frac=pct)])
dirty_test  = pd.concat([dirty_test,   dirty_test.sample(frac=pct)])

# For each row, 0.1% of the time, zero out the image array
dirty_train['image'] = dirty_train['image'].apply(lambda x: np.zeros((size, size)) if np.random.rand() < pct else x)
dirty_test['image']  =  dirty_test['image'].apply(lambda x: np.zeros((size, size)) if np.random.rand() < pct else x)

# Add a column for classification scores from a previous model
dirty_train['predict'] = np.random.normal(0.75, 0.1, dirty_train.shape[0])
dirty_test['predict']  = np.random.normal(0.75, 0.1, dirty_test.shape[0])

# For each row, 0.1% of the time, replace the predict column with a normal distribution centered on 0.25
dirty_train['predict'] = dirty_train['predict'].apply(lambda x: np.random.normal(0.25, 0.1) if np.random.rand() < pct else x)
dirty_test['predict']  =  dirty_test['predict'].apply(lambda x: np.random.normal(0.25, 0.1) if np.random.rand() < pct else x)

# For each row, 0.1% of the time, add/subtract 1 to the predict column
dirty_train['predict'] = dirty_train['predict'].apply(lambda x: x + 1 if np.random.rand() < pct/2 else x)
dirty_test['predict']  =  dirty_test['predict'].apply(lambda x: x + 1 if np.random.rand() < pct/2 else x)
dirty_train['predict'] = dirty_train['predict'].apply(lambda x: x - 1 if np.random.rand() < pct/2 else x)
dirty_test['predict']  =  dirty_test['predict'].apply(lambda x: x - 1 if np.random.rand() < pct/2 else x)

# For each row, 0.1% of the time, choose a column at random and set it to NaN
dirty_train = dirty_train.apply(lambda row: row if np.random.rand() > pct else row.apply(lambda x: np.nan if np.random.rand() > 0.5 else x), axis=1)
dirty_test  =  dirty_test.apply(lambda row: row if np.random.rand() > pct else row.apply(lambda x: np.nan if np.random.rand() > 0.5 else x), axis=1)

# Mislabel 0.1% of the data with strings that look like labels or missing values (e.g., names like "one", numbers greater than 62, " ")
# Create a list of bad labels
bad_labels = ['number', 'letter', 'maybe three?', 'think this is a seven', ' ', '', 'NaaN', 'Null'] + list(range(63, 100))
# For 0.1% of the rows, randomly choose a bad label
dirty_train['label'] = dirty_train['label'].apply(lambda x: np.random.choice(bad_labels) if np.random.rand() < pct else x)
dirty_test['label']  =  dirty_test['label'].apply(lambda x: np.random.choice(bad_labels) if np.random.rand() < pct else x)




# TEMPLATE: For each row, PCT% of the time, randomly apply METHOD() to the COLUMN
# df['COLUMN'] = dirty_train['column'].apply(lambda x: METHOD(x) if np.random.rand() < PCT else x)

# Not applying these for now, but they're here if you want to try them

# For each row, randomly decide whether to apply a random noise
# dirty_train['image'] = dirty_train['image'].apply(lambda x: x + np.random.rand(size, size) if np.random.rand() < 0.1 else x)

# For each row, randomly decide whether to flip the image horizontally
#dirty_train['image'] = dirty_train['image'].apply(lambda x: np.flip(x, axis=1) if np.random.rand() < 0.1 else x)


# Exercise

The changes above were applied randomly, so we'll need to find them and make a plan to fix them.

- [x] Create a column to identify whether each row came from *train* or *test*
- [x] (optional) Merge the data into a single
- [x] Explore the data to understand what's in it
- [x] List potential data issues to fix
    - Duplicate rows
    - Missing values
    - Outliers and out-of-bounds issues
    - Label issues
    - Image issues
    - Zeroed values
- [x] Create a friendlier column for image labels
- [x] Recategorizing the labels into 'numbers' and 'letters'
- [x] Bin the prediction scores of the previous model

In [ ]:
# Let's start cleaning!

# Labels! They're hard to understand as numbers, so let's map them to characters
# We can do this by manually creating a dictionary:
LABELS = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
          'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
          'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

# Or generate the list of labels using the following code:
# create the characters list, which is the digits, then uppercase, then lowercase
chars = string.digits + string.ascii_uppercase + string.ascii_lowercase
# create the dictionary mapping the numbers to the characters
num_to_char = {i: chars[i] for i in range(len(chars))}

# Add a column showing which split (train vs test) each row came from
raw_train['split'] = 'train'
raw_test['split']  = 'test'
dirty_train['split'] = 'train'
dirty_test['split'] = 'test'

# Add a column for a hash of the images (might make it easier to compare them)
dirty_train['image_hash'] = dirty_train['image'].apply(lambda x: sha1(np.ascontiguousarray(x)).hexdigest())
dirty_test['image_hash']  =  dirty_test['image'].apply(lambda x: sha1(np.ascontiguousarray(x)).hexdigest())

## Merge Data

In [ ]:
merged = pd.concat([dirty_test, dirty_train], axis=0)

### Explore merged data
- Use `.info()` to get a general idea of shape and column types
- Use `print()` to visualize first 5 and last 5 rows

In [ ]:
merged.info()

In [ ]:
print(merged)

### Explore columns

#### `label` column
Notes for cleaning: 
- Some bad labels are present
- Solution: 
    - change the bad labels to NaN
    - flag them in another column

In [ ]:
merged['label'].unique()

#### `image` column

In [ ]:
print("The length of one cell from the image column:", len(merged['image'][0]))
print("Sample image cell at row 0:")
print(merged['image'][0])

#### `image_hash` column

In [ ]:
merged['image_hash'].describe()


#### `predict` column

In [ ]:
# Print summary of `predict` variable 
merged['predict'].describe()


Looking at the max and min values, negative values and values over 1 are probably not valid for a probability value. 

In [ ]:
# Weird values for a predicted proportion
print('Number of values over 1:', sum(merged['predict']>1))
print('Number of values less than 0:', sum(merged['predict']<0))
print('Number of values out-of-range:', sum((merged['predict']<0)|(merged['predict']>1)))

# Calculating number of outliers (after deleting values out-of-range)
mask = (0<=merged['predict']) & (merged['predict']<=1)
desc_predict = merged[mask]['predict'].describe()
IQR = desc_predict['75%']-desc_predict['25%']
lower_bound = desc_predict['25%'] - 1.5 *IQR
upper_bound = desc_predict['75%'] + 1.5 *IQR
print('Number of outliers:', sum((merged[mask]['predict']>upper_bound) | (merged[mask]['predict']<lower_bound)))


#### `split` column

In [ ]:
merged['split'].describe()

## Cleaning Data

In [ ]:
# Copy `merged` data frame into a new data frame `work_merged` to work with
work_merged = merged.copy()

#### Clean non-sensible `label`

In [ ]:
# Replace `label` that do not make sense with NaN
for lab in work_merged['label']: 
    if lab not in set(range(0,62)): 
        work_merged['label'].replace(lab, np.nan, inplace=True)

#### Replace out-of-range `predict` values with `NaN`

In [ ]:
print('Total number of non-null `predict` values:', work_merged['predict'].count())

In [ ]:
# Set out-of-range condition as `predict`<0 and `predict`>1
out_condition = (work_merged['predict']<0) | (work_merged['predict']>1)
# Located out-of-range values and replace with NaN
work_merged.loc[out_condition, 'predict']=np.nan
print('Total number of non-null `predict` values after deleting out-of-range values:', work_merged['predict'].count())

#### Replace outlier `predict` values to `NaN`

In [ ]:
desc_predict = work_merged['predict'].describe()
IQR = desc_predict['75%']-desc_predict['25%']
lower_bound = desc_predict['25%'] - 1.5 *IQR
upper_bound = desc_predict['75%'] + 1.5 *IQR
outlier = (work_merged['predict']>upper_bound) | (work_merged['predict']<lower_bound)
# Located outliers and replace with NaN
work_merged.loc[outlier, 'predict']=np.nan
print('Total number of non-null `predict` values after deleting outliers:', work_merged['predict'].count())

#### Clean zero-ed out `image` arrays

In [ ]:
zeroed = work_merged['image'].apply(lambda x: np.all(x==0))
# Locate and replace zeroed images with NaN
work_merged.loc[zeroed, 'image']=np.nan



#### Check for duplicates in rows that do not contain `NaN` values across all columns

In [ ]:
Non_null = ~work_merged.isna().any(axis=1)
clean_merged = work_merged[Non_null]
# Check again that `count()` for number of non-null values across columns in `work_merged[Non_null]` are the same
clean_merged.count()

In [ ]:
# Save unique values list as a new column in `clean_merged`
clean_merged['unique'] = ~clean_merged.duplicated(subset=['image_hash','label','split'], keep='first')
# Generate final cleaned dataframe `final_merged` 
final_merged = clean_merged[clean_merged['unique']==True][['label','image','image_hash', 'predict', 'split']]

#### Check info of `final_merged`

In [ ]:
print(final_merged.info())
print(final_merged.shape)


## Relabeling and binning

#### Create a friendlier column for image labels

In [ ]:
# Copied from above
# We can do this by manually creating a dictionary:
LABELS = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
          'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
          'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

# Or generate the list of labels using the following code:
# create the characters list, which is the digits, then uppercase, then lowercase
chars = string.digits + string.ascii_uppercase + string.ascii_lowercase
# create the dictionary mapping the numbers to the characters
num_to_char = {i: chars[i] for i in range(len(chars))}

# Start of solution: Create a friendlier column for image labels
# Mapping labels and creating a `friendly_label` column
final_merged['friendly_label'] = final_merged['label'].map(num_to_char)
final_merged[['label','friendly_label']]

#### Recategorizing the labels into 'numbers' and 'letters'

In [ ]:
# Function to check string
def check_string_type(str):
    if str.isdigit():
        return 'number'
    elif str.isascii():
        return 'letter'
    
#Apply function to `friendly_label` column
final_merged['label_type']=final_merged['friendly_label'].apply(check_string_type)

# Print 5 rows from each type of column 
print(final_merged[final_merged['label_type']=='number'].head(5))
print(final_merged[final_merged['label_type']=='letter'].head(5))

#### Bin the prediction scores of the previous model

In [ ]:
final_merged['predict'].describe()

In [ ]:
#bin according to quadrants
final_merged['predict_groups'] = pd.qcut(final_merged['predict'], q=4, labels =['very low', 'low', 'medium', 'high'])
final_merged[['predict', 'predict_groups']]